In [4]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


In [7]:
from dotenv import load_dotenv
import os
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

# 1. 응답 스키마 정의
# StructuredOutputParser는 ResponseSchema를 사용해 추출할 정보의 구조를 정의합니다.
response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="여행 만족도 (1-5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트")
]

# 2. 출력 파서와 프롬프트 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

prompt_template = """
주어진 여행 후기 텍스트에서 다음 정보를 추출해줘.

{format_instructions}

여행 후기:
{text}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# 3. 모델 설정
# 환경 변수에 OPENAI_API_KEY를 설정해야 합니다.
# 실제 사용 시, 이 부분을 환경에 맞게 조정해야 합니다.
try:
    api_key = os.environ["OPENAI_API_KEY"]
except KeyError:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0  # 구조화된 데이터 추출에는 낮은 온도가 더 적합합니다.
)

# 4. 체인 구성 및 실행
# 파서의 지시사항을 프롬프트에 주입
chain = prompt | llm | parser

travel_review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 5. 체인 호출 및 결과 출력
structured_info = chain.invoke({"text": travel_review, "format_instructions": format_instructions})

print("--- 추출된 여행 정보 ---")
print(f"여행지: {structured_info['destination']}")
print(f"기간: {structured_info['duration']}")
print(f"예산: {structured_info['budget']}")
print(f"추천도: {structured_info['rating']}")
print(f"주요 활동: {''.join(structured_info['activities'])}")

print("\n--- 결과 타입 ---")
print(type(structured_info))

--- 추출된 여행 정보 ---
여행지: 부산
기간: 2박 3일
예산: 30만원
추천도: 4점
주요 활동: 해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경

--- 결과 타입 ---
<class 'dict'>
